# Assignment 5

For this assignment we were tasked with answering these three questions:

1. What is the northernmost airport in the United States? 
Wiley Post Will Rogers Memorial

2. What is the easternmost airport in the United States? 
Eastport Municipal 

3. On February 12th, 2013, which New York area airport had the windiest weather? 
Laguardia

To begin I took the csv files from the provided github repository. The I read them into pandas dataframe objects for analysis. 

In [125]:
import numpy as np
import pandas as pd
import datetime

df_airlines = pd.read_csv('airlines.csv')
df_airports = pd.read_csv('airports.csv')
df_planes = pd.read_csv('planes.csv')
df_weather = pd.read_csv('weather.csv')

Next I'm going to take a look at each dataframe to see what calculations I need to do to answer the questions. 

In [126]:
df_airlines.head()

  carrier                    name
0      9E       Endeavor Air Inc.
1      AA  American Airlines Inc.
2      AS    Alaska Airlines Inc.
3      B6         JetBlue Airways
4      DL    Delta Air Lines Inc.

In [127]:
df_airports.head()

   faa                           name        lat        lon   alt  tz dst  \
0  04G              Lansdowne Airport  41.130472 -80.619583  1044  -5   A   
1  06A  Moton Field Municipal Airport  32.460572 -85.680028   264  -6   A   
2  06C            Schaumburg Regional  41.989341 -88.101243   801  -6   A   
3  06N                Randall Airport  41.431912 -74.391561   523  -5   A   
4  09J          Jekyll Island Airport  31.074472 -81.427778    11  -5   A   

              tzone  
0  America/New_York  
1   America/Chicago  
2   America/Chicago  
3  America/New_York  
4  America/New_York  

In [128]:
df_planes.head()

  tailnum    year                     type      manufacturer      model  \
0  N10156  2004.0  Fixed wing multi engine           EMBRAER  EMB-145XR   
1  N102UW  1998.0  Fixed wing multi engine  AIRBUS INDUSTRIE   A320-214   
2  N103US  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   A320-214   
3  N104UW  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   A320-214   
4  N10575  2002.0  Fixed wing multi engine           EMBRAER  EMB-145LR   

   engines  seats  speed     engine  
0        2     55    NaN  Turbo-fan  
1        2    182    NaN  Turbo-fan  
2        2    182    NaN  Turbo-fan  
3        2    182    NaN  Turbo-fan  
4        2     55    NaN  Turbo-fan  

In [129]:
df_weather.head()

  origin  year  month  day  hour  ...  wind_gust  precip  pressure  visib  \
0    EWR  2013      1    1     1  ...        NaN     0.0    1012.0   10.0   
1    EWR  2013      1    1     2  ...        NaN     0.0    1012.3   10.0   
2    EWR  2013      1    1     3  ...        NaN     0.0    1012.5   10.0   
3    EWR  2013      1    1     4  ...        NaN     0.0    1012.2   10.0   
4    EWR  2013      1    1     5  ...        NaN     0.0    1011.9   10.0   

              time_hour  
0  2013-01-01T06:00:00Z  
1  2013-01-01T07:00:00Z  
2  2013-01-01T08:00:00Z  
3  2013-01-01T09:00:00Z  
4  2013-01-01T10:00:00Z  

[5 rows x 15 columns]

What we have discovered is that the airports data frame provides us with latitutde and longitude coordinates which we can use to find out the answer to our first question. 

Before we do that though we should clean up our data a bit. Let's see if we have any missing data is the airports dataframe. We will use the isnull() and sum() methods to add up anything. 

In [130]:
df_airports.isnull().sum()

faa      0
name     0
lat      0
lon      0
alt      0
tz       0
dst      0
tzone    3
dtype: int64

We are only missing a few timezones so we should have no problem calculating the northern and eastern most airports. 

Our northern most airports should be the one which has the highest latitudinal value. That's is because the higher the value is the further north it is located. 

In [131]:
northairport = df_airports.lat.idxmax()
df_airports.loc[northairport]

faa                          EEN
name     Dillant Hopkins Airport
lat                    72.270833
lon                    42.898333
alt                          149
tz                            -5
dst                            A
tzone                        NaN
Name: 417, dtype: object

Unfortunately we have an empty value where we should have had the timezone labeled. Without a timezone it's difficult to tell if this airport is in the U.S. or not so we will have to make sure. 

We will do that my matching the tz -5 value with other tz values in our data frame. 

In [132]:
df_airports[df_airports.tz == -5].head()

   faa                            name        lat        lon   alt  tz dst  \
0  04G               Lansdowne Airport  41.130472 -80.619583  1044  -5   A   
3  06N                 Randall Airport  41.431912 -74.391561   523  -5   A   
4  09J           Jekyll Island Airport  31.074472 -81.427778    11  -5   A   
5  0A9  Elizabethton Municipal Airport  36.371222 -82.173417  1593  -5   A   
6  0G6         Williams County Airport  41.467306 -84.506778   730  -5   A   

              tzone  
0  America/New_York  
3  America/New_York  
4  America/New_York  
5  America/New_York  
6  America/New_York  

This would seem to confirm that it is indeed the northern most airport. However, a little research tells us that almost all U.S. locations east of alaska are below the 49th paralell. In fact plugging this airport's coordinates into google maps gives us a location which is in the sea somewhere north of the scandanavian penninsula. 

We will not lose heart though. We will build on our research and assume that the northern most airport must be somewhere in Alaska. So we will narrow our data down to only airports located in the American/Anchorage timezone. 

In [133]:
alaskaair = df_airports[df_airports.tz == -9]
northern = alaskaair.lat.idxmax()
alaskaair.loc[northern]

faa                             BRW
name     Wiley Post Will Rogers Mem
lat                       71.285446
lon                     -156.766003
alt                              44
tz                               -9
dst                               A
tzone             America/Anchorage
Name: 230, dtype: object

This give us our answer which we confirmed with a quick google. 

The answer to question 1 is: 

What is the nortern most airport in the United States?
Wiley Post Will Rogers Memorial Airport

Lets try the same method to find the easternmost airport. Postitive longitudes are east and negative are west so we will again just look for the max value. 

In [134]:
eastairport = df_airports.lon.idxmax()
df_airports.loc[eastairport]

faa                    SYA
name          Eareckson As
lat              52.712275
lon              174.11362
alt                     98
tz                      -9
dst                      A
tzone    America/Anchorage
Name: 1290, dtype: object

This airport is an interesting case. When we look at it on the map it would seem like it was actually the western most airport in the U.S. that is because it is so far west that it's coordinates are in the positive again. 

We will use a similar solution to this question as we did for the previous one and assume that the eastern most airport has to be somewhere in the New York time zone. Most of the longitudes in the U.S. are negative but the one that is closest to zero should still be the one furtherest east and so we will still be looking for the max value. 

In [135]:
newyorkair = df_airports[df_airports.tz == -5]
eastern = newyorkair.lon.idxmax()
newyorkair.loc[eastern]

faa                          EEN
name     Dillant Hopkins Airport
lat                    72.270833
lon                    42.898333
alt                          149
tz                            -5
dst                            A
tzone                        NaN
Name: 417, dtype: object

There is the pesky Dillant Hopkins Airport again. Further research tells us that the latitude and longitude values of this airport are actually reversed. So we'll fix that so we can finally get our answer. 

In [136]:
df_airports1 = df_airports.replace({'lat': 72.270833}, 42.898333)
df_airports2 = df_airports1.replace({'lon': 42.898333}, -72.270833)
df_airports2.loc[417]

faa                          EEN
name     Dillant Hopkins Airport
lat                    42.898333
lon                   -72.270833
alt                          149
tz                            -5
dst                            A
tzone                        NaN
Name: 417, dtype: object

now that we have this airport located correctly let's try that again. 

In [137]:
newyorkair = df_airports2[df_airports2.tz == -5]
eastern = newyorkair.lon.idxmax()
newyorkair.loc[eastern]

faa                             EPM
name     Eastport Municipal Airport
lat                       44.910111
lon                      -67.012694
alt                              45
tz                               -5
dst                               A
tzone              America/New_York
Name: 444, dtype: object

Answer to question 2:

What is the easternmost airport in the United States? 
Eastport Municipal Airport 

Only one more question to go. We will be looking for weather conditions for three specific airports on a specific date. We will quickly check to make sure the data in our dataframe is suitable for analysis. 

In [138]:
df_weather.isnull().sum()

origin           0
year             0
month            0
day              0
hour             0
             ...  
wind_gust    20778
precip           0
pressure      2729
visib            0
time_hour        0
Length: 15, dtype: int64

This shows us that there are many missing values in the wind gust column. We will have to be wary of this when performing our analysis. 

First we need to separate out our data into three separate dataframes for all the hours of the day at each airport. We will then use our mathematical functions to calculate which airport had the windiest conditions. 

In [139]:
ewr = df_weather[(df_weather.origin == 'EWR') & 
                 (df_weather.year == 2013) & 
                 (df_weather.month == 2) & 
                 (df_weather.day == 12)]
ewr

     origin  year  month  day  hour  ...  wind_gust  precip  pressure  visib  \
1006    EWR  2013      2   12     0  ...        NaN     0.0    1006.9   10.0   
1007    EWR  2013      2   12     1  ...        NaN     0.0    1007.2   10.0   
1008    EWR  2013      2   12     2  ...   25.31716     0.0    1007.8   10.0   
1009    EWR  2013      2   12     3  ...        NaN     0.0    1008.3   10.0   
1010    EWR  2013      2   12     4  ...        NaN     0.0    1008.8   10.0   
...     ...   ...    ...  ...   ...  ...        ...     ...       ...    ...   
1025    EWR  2013      2   12    19  ...        NaN     0.0    1015.0   10.0   
1026    EWR  2013      2   12    20  ...        NaN     0.0    1013.9   10.0   
1027    EWR  2013      2   12    21  ...        NaN     0.0    1013.0   10.0   
1028    EWR  2013      2   12    22  ...        NaN     0.0    1013.4   10.0   
1029    EWR  2013      2   12    23  ...        NaN     0.0    1013.7   10.0   

                 time_hour  
1006  2013

In [140]:
jfk = df_weather[(df_weather.origin == 'JFK') & 
                 (df_weather.year == 2013) & 
                 (df_weather.month == 2) & 
                 (df_weather.day == 12)]
jfk

     origin  year  month  day  hour  ...  wind_gust  precip  pressure  visib  \
9709    JFK  2013      2   12     0  ...        NaN     0.0    1007.1   10.0   
9710    JFK  2013      2   12     1  ...        NaN     0.0    1007.3    7.0   
9711    JFK  2013      2   12     2  ...   24.16638     0.0    1007.7   10.0   
9712    JFK  2013      2   12     3  ...   26.46794     0.0    1008.5   10.0   
9713    JFK  2013      2   12     4  ...   21.86482     0.0    1009.0   10.0   
...     ...   ...    ...  ...   ...  ...        ...     ...       ...    ...   
9728    JFK  2013      2   12    19  ...        NaN     0.0    1015.1   10.0   
9729    JFK  2013      2   12    20  ...        NaN     0.0    1014.1   10.0   
9730    JFK  2013      2   12    21  ...        NaN     0.0    1013.3   10.0   
9731    JFK  2013      2   12    22  ...        NaN     0.0    1013.4   10.0   
9732    JFK  2013      2   12    23  ...        NaN     0.0    1014.0   10.0   

                 time_hour  
9709  2013

In [141]:
lga = df_weather[(df_weather.origin == 'LGA') & 
                 (df_weather.year == 2013) & 
                 (df_weather.month == 2) & 
                 (df_weather.day == 12)]
lga

      origin  year  month  day  hour  ...  wind_gust  precip  pressure  visib  \
18415    LGA  2013      2   12     0  ...        NaN     0.0       NaN   10.0   
18416    LGA  2013      2   12     1  ...   28.76950     0.0    1006.6   10.0   
18417    LGA  2013      2   12     2  ...   31.07106     0.0    1007.1   10.0   
18418    LGA  2013      2   12     3  ...   28.76950     0.0    1007.9   10.0   
18419    LGA  2013      2   12     4  ...        NaN     0.0    1008.3   10.0   
...      ...   ...    ...  ...   ...  ...        ...     ...       ...    ...   
18434    LGA  2013      2   12    19  ...   21.86482     0.0    1014.7   10.0   
18435    LGA  2013      2   12    20  ...        NaN     0.0    1013.6   10.0   
18436    LGA  2013      2   12    21  ...        NaN     0.0    1012.8   10.0   
18437    LGA  2013      2   12    22  ...        NaN     0.0    1012.7   10.0   
18438    LGA  2013      2   12    23  ...        NaN     0.0    1013.5   10.0   

                  time_hour

Now the best method is probably get the mean value for wind gust speeds at each airport and see which has the highest value. 

In [142]:
ewr.wind_gust.mean()

25.1733125

In [143]:
jfk.wind_gust.mean()

24.58484545454546

In [144]:
lga.wind_gust.mean()

24.885617500000006

At first glance it would seem like Newark had the windiest conditions on this date, but we need to check our data for outliers to make sure. 

In [145]:
ewr.wind_gust.isnull().sum()

16

In [146]:
jfk.wind_gust.isnull().sum()

13

In [147]:
lga.wind_gust.isnull().sum()

8

By checking for null values we see that Newark actually has the highest number of blank spaces in it's data set which means that although there may have been brief periods of strong wind gust it was actually a pretty calm day in New Jersey. The smaller number of values is adding up to a higher average wind rate. What we are going to do is assume that the blank values in the data are wind speeds of zero. Using pandas we will replace them this way. 

In [157]:
ewr_wind = ewr.wind_gust.fillna(0)
ewr_wind

1006     0.00000
1007     0.00000
1008    25.31716
1009     0.00000
1010     0.00000
          ...   
1025     0.00000
1026     0.00000
1027     0.00000
1028     0.00000
1029     0.00000
Name: wind_gust, Length: 24, dtype: float64

In [159]:
jfk_wind = jfk.wind_gust.fillna(0)
jfk_wind

9709     0.00000
9710     0.00000
9711    24.16638
9712    26.46794
9713    21.86482
          ...   
9728     0.00000
9729     0.00000
9730     0.00000
9731     0.00000
9732     0.00000
Name: wind_gust, Length: 24, dtype: float64

In [160]:
lga_wind = lga.wind_gust.fillna(0)
lga_wind

18415     0.00000
18416    28.76950
18417    31.07106
18418    28.76950
18419     0.00000
           ...   
18434    21.86482
18435     0.00000
18436     0.00000
18437     0.00000
18438     0.00000
Name: wind_gust, Length: 24, dtype: float64

Our mean values should now more accurately represent the wind conditions. 

In [165]:
ewr_wind.mean()

8.391104166666667

In [166]:
jfk_wind.mean()

11.268054166666667

In [168]:
lga_wind.mean()

16.59041166666667

So we now see that LaGuardia had the stongest average wind gust speed. Combining that with our knowledge that this airport also had the highest number of recordable windgusts, we can be relatively certain that this airport had the windiest conditions. 

Answer to question 3:

On February 12th, 2013, which New York area airport had the windiest weather?
LaGuardia